<a href="https://colab.research.google.com/github/stef4k/train-maintenance-data-mining/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification


In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from collections import Counter
import ast


from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from gensim.models import Word2Vec
from scipy.optimize import minimize
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tqdm import tqdm
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")


Manually remove the first ';' from the first row in csv file

In [2]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';', index_col=0)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
195,4441897,"[536, 536, 536, 536, 536, 536, 536, 536, 536, ...","[4026, 4016, 4020, 4026, 4016, 4114, 4168, 414...","[-13431, -13430, -13423, -13423, -9161, -9144,...",50.557854,4.471380,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Fal...",4
717,4466199,"[1022, 1022, 1022, 1022, 1022, 1022, 1022, 102...","[4066, 4066, 4068, 4068, 4066, 4068, 4066, 406...","[-13611, -13280, -12866, -12489, -12288, -1165...",50.522553,5.544939,"[0.2, 0.6, 0.2, 0.5, 0.4, 0.1, 1.8, 0.5, 0.3, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",99


In [3]:
class Word2VecVectorizer(TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.w2v_model = None

    def fit(self, X, y=None):
        sentences = [sentence.split() for sentence in X]
        self.w2v_model = Word2Vec(sentences, vector_size=self.size, window=self.window,
                                  min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X, y=None):
        transformed_data = np.array([
            np.mean([self.w2v_model.wv[word] for word in sentence.split() if word in self.w2v_model.wv]
                    or [np.zeros(self.
                                 size)], axis=0)
            for sentence in X
        ])
        return csr_matrix(transformed_data)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [4]:

class EnsembleModel:
    def __init__(self, trained_models, trained_vectorizers):

        self.trained_models = trained_models
        self.trained_vectorizers = trained_vectorizers
        self.optimized_weights = None

    def _generate_prediction_matrix(self, X):
        predictions = {}
        for (vect_name, samp_name, clf_name), model in self.trained_models.items():
            vectorizer = deepcopy(self.trained_vectorizers[vect_name])
            X_vect = vectorizer.transform(X).toarray()

            # Collect probability predictions for multiclass
            predictions[(vect_name, samp_name, clf_name)] = model.predict_proba(X_vect)

        # Convert predictions to a 3D array: (num_samples, num_classes, num_models)
        return np.stack(list(predictions.values()), axis=2)

    def fit(self, X_train, y_train):
        pred_matrix = self._generate_prediction_matrix(X_train)

        # Define fitness function for genetic algorithm
        def fitness(weights):
            # Apply weights to model predictions
            weighted_pred = np.tensordot(pred_matrix, weights, axes=([2], [0]))
            # Get the predicted class with the highest weighted probability
            final_pred = np.argmax(weighted_pred, axis=1)
            # Return negative weighted F1 score for optimization
            return -f1_score(y_train, final_pred, average='weighted')

        # Genetic algorithm for weight optimization
        num_models = pred_matrix.shape[2]
        bounds = [(0, 1)] * num_models
        result = differential_evolution(fitness, bounds)

        # Store optimized weights
        self.optimized_weights = result.x

    def predict(self, X):
        if self.optimized_weights is None:
            raise ValueError("The ensemble model must be trained using `train_ensemble` before prediction.")

        # Generate predictions for the input data
        pred_matrix = self._generate_prediction_matrix(X)
        weighted_pred = np.tensordot(pred_matrix, self.optimized_weights, axes=([2], [0]))
        ensemble_pred = np.argmax(weighted_pred, axis=1)
        return ensemble_pred


In [5]:

class Representations:
  def __init__(self, df):
    self.df = df

  def representation_a(self, df):
            events_types_dict = {}
            for events_sequence in df['events_sequence']:
                row_list = ast.literal_eval(events_sequence)  # Transforming string into actual list
                unique_events = set(row_list)
                for event in unique_events:
                    if not events_types_dict.get(event):
                        events_types_dict[event] = 0
                    events_types_dict[event] += 1

            # Step 2: Sort events by frequency and calculate percentages
            sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
            sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
            sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / df.shape[0] * 100
            sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)  # Ensure event_type is string

            # Step 3: Filter events with percentage <= 85%
            events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage <= 85].event_type)))

            # Step 4: Clean the 'events_sequence' column
            df['clean_events_sequence'] = (
                df['events_sequence']
                .apply(ast.literal_eval)  # Convert string to list
                .apply(lambda x: [i for i in x if i in events_low_frequency])  # Filter low-frequency events
                .astype(str)
                .replace(r'[\[\],]', '', regex=True)  # Remove brackets and commas
            )

            # Step 5: Extract the target column
            self.y = df['incident_type'].copy()  # Separate target column
            self.X = df['clean_events_sequence'].copy()
            return self.X, self.y

  def representation_b(self, df):

    before_incident = []
    after_incident = []

    for _, row in df.iterrows():
        # Convertir las cadenas a listas
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        # Filtrar eventos antes del incidente (t <= 0)
        before_events = " ".join([str(event) for event, time in zip(events, seconds) if time <= 0])
        if before_events:
            before_incident.append({
                "events_sequence": before_events,
                "class": incident_type
            })

        # Filtrar eventos después del incidente (t > 0)
        after_events = " ".join([str(event) for event, time in zip(events, seconds) if time > 0])
        if after_events:
            after_incident.append({
                "events_sequence": after_events,
                "class": 100
            })

    # Convertir listas a DataFrames
    before_df = pd.DataFrame(before_incident)
    after_df = pd.DataFrame(after_incident)

    return before_df, after_df


  def representation_c(df, sequence_length=30):

    overlapping_sequences = []
    step = sequence_length // 2  # Paso entre ventanas solapadas

    for _, row in df.iterrows():
        # Convertir eventos y tiempos en listas
        events = ast.literal_eval(row['events_sequence'])
        seconds = ast.literal_eval(row['seconds_to_incident_sequence'])
        incident_type = row['incident_type']

        # Generar cadenas antes del incidente
        for i in range(0, len(events) - sequence_length + 1, step):
            sequence = events[i:i + sequence_length]
            seconds_slice = seconds[i:i + sequence_length]

            # Verificar si la mayoría de los tiempos son antes o después del incidente
            if all(sec <= 0 for sec in seconds_slice):
                sequence_class = incident_type
            elif all(sec > 0 for sec in seconds_slice):
                sequence_class = 100
            else:
                # Mezcla de antes y después del incidente, no considerar esta secuencia
                continue

            overlapping_sequences.append({"sequence": sequence, "class": sequence_class})

    # Crear un nuevo DataFrame con las secuencias generadas
    sequences_df = pd.DataFrame(overlapping_sequences)
    return sequences_df


In [28]:

class Experiment:
    def __init__(self, X, y):
        self.X = X
        self.le = LabelEncoder()
        self.y = self.le.fit_transform(y)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
        self.y_train = self.le.transform(self.y_train)
        self.y_test = self.le.transform(self.y_test)
        self.trained_models = {}
        self.trained_vectorizers = {}
        self.trainer_samplers = {}
        self.results = []
        self.sampling_strategies = {
            "SMOTE": SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            #"Borderline-SMOTE": BorderlineSMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            #"ADASYN": ADASYN(sampling_strategy='auto', random_state=1, n_neighbors=3),
            #"RandomOversampler": RandomOverSampler(sampling_strategy='auto', random_state=1),
            #"SMOTE-ENN": SMOTEENN(sampling_strategy='auto', random_state=1),
            #"SMOTE-Tomek": SMOTETomek(sampling_strategy='auto', random_state=1)
        }

        self.vectorizers = {
            #"TFIDF": TfidfVectorizer(),
            "Count": CountVectorizer(),
            #"Word2Vec": Word2VecVectorizer(size=100, window=5, min_count=1)
        }
        self.classifiers = {
            'LogisticRegression': LogisticRegression(),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GaussianNB': GaussianNB(),
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(probability=True),
            'XGBoost': XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False),
        }


    def duplicate_minor_classes(self, X, y, min_instances=5):
        X = X.reset_index(drop=True)
        y  = y.reset_index(drop=True)
        class_counts = y.value_counts()

        # Identificar las clases minoritarias
        minor_classes = class_counts[class_counts < min_instances].index

        # Filtrar las filas correspondientes a las clases minoritarias
        minor_class_rows = X.loc[y.isin(minor_classes)]
        minor_class_labels = y.loc[y.isin(minor_classes)]

        # Duplicar las filas y etiquetas correspondientes
        duplicated_X = pd.concat([minor_class_rows] * 2, ignore_index=True)
        duplicated_y = pd.concat([minor_class_labels] * 2, ignore_index=True)

        # Combinar con las filas y etiquetas originales
        X_balanced = pd.concat([X, duplicated_X], ignore_index=True)
        y_balanced = pd.concat([y, duplicated_y], ignore_index=True)

        X_balanced.reset_index(drop=True, inplace=True)
        y_balanced.reset_index(drop=True, inplace=True)

        return X_balanced, y_balanced


    def test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name):
        """Test a model with Stratified K-Fold and return an array with metric results."""
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        accuracies, recalls, precisions, f1s = [], [], [], []
        X, y = self.X_train, self.y_train

        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            X_train = vectorizer.fit_transform(X_train).toarray()
            X_test = vectorizer.transform(X_test).toarray()

            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

            model.fit(X_resampled, y_resampled)

            y_pred = model.predict(X_test)

            # Collect metrics for each fold
            accuracies.append(accuracy_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred, average='weighted'))
            precisions.append(precision_score(y_test, y_pred, average='weighted'))
            f1s.append(f1_score(y_test, y_pred, average='weighted'))

        # Return average metrics as a result array
        return [
            model_name, vectorizer_name, sampler_name,
            np.mean(accuracies), np.std(accuracies),
            np.mean(recalls), np.std(recalls),
            np.mean(precisions), np.std(precisions),
            np.mean(f1s), np.std(f1s)
        ]

    def training(self):
        results = []
        progress_bar = tqdm(total = len(self.vectorizers.keys()) * len(self.sampling_strategies.keys()) * len(self.classifiers.keys()))
        self.X_train, self.y_train = self.duplicate_minor_classes(self.X_train, pd.Series(self.y_train))
        # Iterate over vectorizers, samplers, and classifiers
        for vect_name, ovectorizer in self.vectorizers.items():
            vectorizer = deepcopy(ovectorizer)
            for samp_name, osampler in self.sampling_strategies.items():
                sampler = deepcopy(osampler)
                for clf_name, omodel in self.classifiers.items():
                    model = deepcopy(omodel)

                    result = self.test(
                        model=model,
                        model_name=clf_name,
                        vectorizer=vectorizer,
                        vectorizer_name=vect_name,
                        sampler=sampler,
                        sampler_name=samp_name
                    )
                    # Store the trained model and add result to the results list
                    results.append(result)

                    vectorizer = deepcopy(ovectorizer)
                    sampler = deepcopy(osampler)
                    model = deepcopy(omodel)
                    vectorizer.fit(self.X_train)
                    X_train = vectorizer.transform(self.X_train).toarray()
                    X_resampled, y_resampled = sampler.fit_resample(X_train, self.y_train)
                    model.fit(X_resampled, y_resampled)
                    self.trained_models[(vect_name, samp_name, clf_name)] = model
                    self.trained_vectorizers[vect_name] = vectorizer
                    self.trainer_samplers[(vect_name, samp_name)] = sampler
                    progress_bar.update(1)

        # Convert results to a DataFrame for better readability
        columns = [
            'Model', 'Vectorizer', 'Sampler',
            'Accuracy Mean', 'Accuracy Std',
            'Recall Mean', 'Recall Std',
            'Precision Mean', 'Precision Std',
            'F1 Mean', 'F1 Std'
        ]
        results_df = pd.DataFrame(results, columns=columns)
        self.results = results_df
        return results_df




In [29]:
X, y = df.events_sequence, df.incident_type
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))


 30%|███       | 3/10 [00:37<01:17, 11.08s/it]

KeyboardInterrupt: 

In [93]:
results

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
0,LogisticRegression,Count,SMOTE,0.471486,0.017767,0.471486,0.017767,0.512338,0.008860,0.469512,0.015478
1,DecisionTree,Count,SMOTE,0.459303,0.032458,0.459303,0.032458,0.471232,0.032722,0.460248,0.032103
2,RandomForest,Count,SMOTE,0.640542,0.031054,0.640542,0.031054,0.648740,0.037938,0.632973,0.032436
3,ExtraTreesClassifier,Count,SMOTE,0.679255,0.037355,0.679255,0.037355,0.683635,0.048037,0.670914,0.042893
4,GradientBoostingClassifier,Count,SMOTE,0.705687,0.006607,0.705687,0.006607,0.714770,0.008536,0.703078,0.007395
5,AdaBoostClassifier,Count,SMOTE,0.481773,0.055515,0.481773,0.055515,0.548591,0.051530,0.493769,0.054542
6,GaussianNB,Count,SMOTE,0.513224,0.022765,0.513224,0.022765,0.575248,0.017215,0.515439,0.016750
7,KNN,Count,SMOTE,0.290267,0.021185,0.290267,0.021185,0.332371,0.022227,0.286779,0.021274
8,SVM,Count,SMOTE,0.311629,0.039142,0.311629,0.039142,0.372378,0.042770,0.303558,0.036071
9,XGBoost,Count,SMOTE,0.696540,0.016580,0.696540,0.016580,0.704206,0.027843,0.693191,0.017171


In [96]:
X, y = Representations(df).representation_a(df)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))


100%|██████████| 10/10 [05:28<00:00, 32.87s/it]


              precision    recall  f1-score   support

           0       0.61      0.83      0.70        24
           1       0.61      0.69      0.65        16
           2       0.71      0.74      0.72        23
           3       0.43      0.60      0.50         5
           4       0.90      0.88      0.89        64
           5       0.73      0.73      0.73        30
           6       0.83      0.54      0.66        35

    accuracy                           0.75       197
   macro avg       0.69      0.72      0.69       197
weighted avg       0.77      0.75      0.75       197



In [97]:
results

,Model,Vectorizer,Sampler,Accuracy Mean,Accuracy Std,Recall Mean,Recall Std,Precision Mean,Precision Std,F1 Mean,F1 Std
0,LogisticRegression,Count,SMOTE,0.626277,0.043505,0.626277,0.043505,0.635346,0.042035,0.626121,0.041839
1,DecisionTree,Count,SMOTE,0.503087,0.022870,0.503087,0.022870,0.525175,0.031072,0.507673,0.022681
2,RandomForest,Count,SMOTE,0.639527,0.032466,0.639527,0.032466,0.638117,0.038609,0.629648,0.033773
3,ExtraTreesClassifier,Count,SMOTE,0.657868,0.028028,0.657868,0.028028,0.664988,0.031401,0.648764,0.028840
4,GradientBoostingClassifier,Count,SMOTE,0.709764,0.017843,0.709764,0.017843,0.718387,0.016190,0.707514,0.019191
5,AdaBoostClassifier,Count,SMOTE,0.495882,0.076081,0.495882,0.076081,0.528046,0.066722,0.496413,0.078922
6,GaussianNB,Count,SMOTE,0.491837,0.016405,0.491837,0.016405,0.565589,0.023900,0.494206,0.013334
7,KNN,Count,SMOTE,0.400186,0.020440,0.400186,0.020440,0.469941,0.022513,0.408985,0.021411
8,SVM,Count,SMOTE,0.484730,0.014635,0.484730,0.014635,0.560749,0.011080,0.483875,0.018098
9,XGBoost,Count,SMOTE,0.675168,0.028482,0.675168,0.028482,0.677742,0.033846,0.668889,0.028916


In [ ]:
df_before, df_after = Representations(df).representation_b(df)
X, y = pd.concat([df_before.events_sequence, df_after.events_sequence]), pd.concat([df_before['class'], df_after['class']])
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))


 20%|██        | 2/10 [01:41<06:07, 45.99s/it]

In [ ]:
results

In [ ]:
X, y = Representations(df).representation_c(df)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))